# Developing JuliaFEM

Author(s): Jukka Aho

**Abstract**: General developer notes.

In [1]:
using Logging
using FactCheck
Logging.configure(level=DEBUG)

Logger(root,DEBUG,PipeEndpoint(open, 0 bytes waiting),root)

## Developing own element

Finite element definition, from [FEniCS-book](https://bitbucket.org/fenics-project/fenics-book/src/7d3a80e7dda0fc279c7964dc6000d57942f11eb3/fenicsbook.cls?at=master) [Ciarlet, 2002]:

-  the domain $T$ is a bounded, closed subset of $\mathbb{R}^d$ (for $d = 1, 2, 3, \dots$) with nonempty interior and piecewise smooth boundary;
- the space $\mathcal{V} = \mathcal{V}(T)$ is a finite dimensional function space on $T$ of dimension $n$;
- the set of degrees of freedom (nodes) $\mathcal{L} = \{\ell_1, \ell_2,\ldots, \ell_{n}\}$ is a basis for the dual space $\mathcal{V}'$; that is, the space of bounded linear functionals on $\mathcal{V}$.

We extend this definition so that domain $T$ can also be empty.

Minimum requirements for element:
- subclass from Element, if not wanting to implement everything by youself
- define basis and partial derivatives of it, because we need to interpolate over it
- give connectivity information, how this element is connected to other elements
- create proper constructor (see example).

Test the element using ``test_element`` function. It it passes, then element implementation should be fine. As an example, we define 4 node quadrilateral element using linear Lagrange basis. We really don't care much how element is implemented as long it's interface is constructed with some rules. The interface is tested using `test_element` and it also gives information how to fix element if something is missing.

In [2]:
using JuliaFEM: Element

Creating element Point1: 1 node point element
Creating element Seg2: 2 node linear line element
Creating Lagrange basis for element Seg2. Number of basis functions: 2. Element dimension: 1
Calculating inverse of A


Here's one basic implementation. The actual element:

In [3]:
type MyQuad4 <: Element
    connectivity :: Array{Int, 1}
    fields :: Dict{Any, Any}
end

Element JuliaFEM.Seg2 created.
Creating element Seg3: 3 node quadratic line element
Creating Lagrange basis for element Seg3. Number of basis functions: 3. Element dimension: 1
Calculating inverse of A
Element JuliaFEM.Seg3 created.
Creating element Quad4: 4 node bilinear quadrangle element
Creating Lagrange basis for element Quad4. Number of basis functions: 4. Element dimension: 2
Calculating inverse of A
Element JuliaFEM.Quad4 created.
Creating element Tet10: 10 node quadratic tetrahedron
Creating Lagrange basis for element Tet10. Number of basis functions: 10. Element dimension: 3
Calculating inverse of A
Element JuliaFEM.Tet10 created.


Default constructor, providing connectivity data needed in assembly

In [4]:
MyQuad4(connectivity) = MyQuad4(connectivity, Dict{Any, Any}())

MyQuad4

Some basic charasteristics like number of basis funcitons and dimension:

In [5]:
JuliaFEM.get_number_of_basis_functions(el::Type{MyQuad4}) = 4
JuliaFEM.get_element_dimension(el::MyQuad4) = 2

get_element_dimension (generic function with 6 methods)

The most important, it's basis (we probably want to interpolate something with this element):

In [6]:
function JuliaFEM.get_basis(el::MyQuad4, xi)
    [(1-xi[1])*(1-xi[2])/4
     (1+xi[1])*(1-xi[2])/4
     (1+xi[1])*(1+xi[2])/4
     (1-xi[1])*(1+xi[2])/4]
end

function JuliaFEM.get_dbasisdxi(el::MyQuad4, xi)
    [-(1-xi[2])/4.0    -(1-xi[1])/4.0
      (1-xi[2])/4.0    -(1+xi[1])/4.0
      (1+xi[2])/4.0     (1+xi[1])/4.0
     -(1+xi[2])/4.0     (1-xi[1])/4.0]
end

get_dbasisdxi (generic function with 6 methods)

Next we check that everything is well defined:

In [7]:
using JuliaFEM: test_element
test_element(MyQuad4)

10-Sep 20:16:49:INFO:root:Testing element MyQuad4
10-Sep 20:16:49:INFO:root:number of basis functions in this element: 4
10-Sep 20:16:49:INFO:root:Initializing element
10-Sep 20:16:49:INFO:root:Element dimension: 2
10-Sep 20:16:49:INFO:root:Setting scalar field [1 2 3 4] to element.
10-Sep 20:16:50:INFO:root:Interpolating scalar field at [0.0,0.0]


PipeEndpoint(open, 0 bytes waiting)

10-Sep 20:16:50:INFO:root:Value: [2.5]
10-Sep 20:16:50:INFO:root:Element MyQuad4 passed tests.


If `test_element` passes, element should be well defined. At least in the sense that it has all necessary things defined ready to be used in JuliaFEM. After building element, one can interpolate things in it. Couple examples:

In [8]:
using JuliaFEM: set_field, interpolate
el1 = MyQuad4([1, 2, 3, 4])
set_field(el1, :temperature, [1 2 3 4])
set_field(el1, :geometry, [0.0 0.0 0.0; 10.0 0.0 0.0; 10.0 1.0 0.0; 0.0 1.0 0.0]');
set_field(el1, :"heat coefficient", 1);

In [9]:
# temperature at the middle poinf of the element, 1/4*(1+2+3+4)
interpolate(el1, :temperature, [0.0, 0.0])

2.5

In [10]:
# geometry midpoint of element
interpolate(el1, :geometry, [0.0, 0.0])

3-element Array{Float64,1}:
 5.0
 0.5
 0.0

In [12]:
# interpolating scalar -> scalar.
interpolate(el1, :"heat coefficient", [0.0, 0.0])

1

## Developing own formulation

Let's consider Poisson equation
\begin{align}
\Delta{u} &= 0 && \text{on } \Omega \\
u &= u_0 && \text{on } \Gamma_{\mathrm{D}} \\
\frac{\partial u}{\partial n} &= g && \text{on } \Gamma_{\mathrm{N}}
\end{align}

Weak form is, find $u\in\mathcal{U}$ such that
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x = \int_{\Gamma_{\mathrm{N}}}g v\,\mathrm{d}s \quad \forall v\in\mathcal{V}.
\end{equation}

Minimum requirements for equation: 
- subclass from Equation, if not want to implement from scratch
- it needs to have lhs and rhs functions
- default constructor takes the element as input argument

Now we have function `test_equation`, which we can use to test that everything is working as expected. 

Again thanks to multiple dispatch, you are free to code your weak form however you want as long as it returns lhs and rhs sides for element dofs. This kind of freedom gives good opportunities to wrap e.g. Fortran code from some other projects. And again we have some suggestions ad following these ideas you get a lot of stuff for free. First we look the left hand side of the equation, that is,
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x
\end{equation}

In [15]:
using JuliaFEM: Equation, IntegrationPoint, Quad4

abstract Heat <: Equation

Our basic data type often looks something like this:

In [16]:
"""
Diffusive heat transfer for 4-node bilinear element.
"""
type DC2D4 <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

We must provide default constructor which takes element as input argument:

In [17]:
function DC2D4(el::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    set_field(el, :temperature, zeros(2, 4))
    DC2D4(el, integration_points, [])
end

DC2D4

Now the actual implementation for $\int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x$:

In [18]:
using JuliaFEM: get_element, get_dbasisdX

"""
Left hand side defined in integration point
"""
function JuliaFEM.get_lhs(eq::DC2D4, ip)
    el = get_element(eq)
    dNdX = get_dbasisdX(el, ip.xi)
    hc = interpolate(el, :"temperature thermal conductivity", ip.xi)
    return dNdX*hc*dNdX'
end
JuliaFEM.has_lhs(eq::DC2D4) = true

has_lhs (generic function with 2 methods)

And that's it. If we want to play with this formulation, we must create element and assign this equation for it:

In [19]:
using JuliaFEM: integrate, integrate_lhs, integrate_rhs
el = Quad4([1, 2, 3, 4])
set_field(el, :geometry, [0 0; 1 0; 1 1; 0 1]')
set_field(el, :"temperature thermal conductivity", 6)
eq = DC2D4(el)
integrate_lhs(eq)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

If rhs or lhs is not defined, integration returns nothing.

In [20]:
isa(integrate_rhs(eq), Void)

true

Next heat flux on boundary:

In [21]:
using JuliaFEM: get_basis, Seg2

"""
Diffusive heat transfer for 2-node linear segment.
"""
type DC2D2 <: Heat
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

function DC2D2(el::Seg2)
    integration_points = [
        IntegrationPoint([0], 2.0)]
    set_field(el, :temperature, zeros(2, 1))
    DC2D2(el, integration_points, [])
end

"""
Right hand side defined in integration point
"""
function JuliaFEM.get_rhs(eq::DC2D2, ip)
    el = get_element(eq)
    N = get_basis(el, ip.xi)
    f = interpolate(el, :"temperature flux", ip.xi)
    return f*N
end
JuliaFEM.has_rhs(eq::DC2D2) = true

has_rhs (generic function with 2 methods)

In [22]:
el = Seg2([1, 2])
set_field(el, :geometry, [0.0 0.0; 0.0 1.0]')
set_field(el, :"temperature flux", 100.0)
eq = DC2D2(el)
integrate_rhs(eq)

2-element Array{Float64,1}:
 50.0
 50.0

## Defining own problem

- takes a set of elements and maps corresponding equations for them
- boundary conditions
- solve!(problem) updates fields

In [ ]:
abstract Problem

type HeatProblem <: Problem
    elements: Array{Any, 1}
    equations: Array{Any, 1}
    boundary_conditions: Array{Any, 1}
end

Developing boundary conditions

In [ ]:
abstract BC

type DefaultBC <: BC
    
end

In [ ]:
bc = NodalBC()
bc[:displacement, [1, 2, 3], 1:2] = 3.0

In [ ]:
bc = WeakBC()
bc[:displacement, [el1, el2, el3]] = (X) -> X[0] + X[1] - 2

In [ ]:
bc = MPCBC()
bc[:displacement, 

In [ ]:
bc = MortarBC()
